In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [ ]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,100))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [7]:
file_path = 'E:\datatest\PseDPC\d7\d71'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
711.csv
**************************************************
2022-02-24 18:33:30.735676
2022-02-24 18:33:35.671455
2022-02-24 18:33:59.583479
##################################################
712.csv
**************************************************
2022-02-24 18:34:11.291153
2022-02-24 18:34:15.714320
2022-02-24 18:34:22.886132
##################################################
713.csv
**************************************************
2022-02-24 18:34:28.127110
2022-02-24 18:34:32.449546
2022-02-24 18:34:53.781472
##################################################
714.csv
**************************************************
2022-02-24 18:35:05.603843
2022-02-24 18:35:09.844495
2022-02-24 18:35:31.049760
##################################################
715.csv
**************************************************
2022-02-24 18:35:45.635736
2022-02-24 18:35:50.212491
2022-02-24 18:36:12.953648
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d7\d72'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
721.csv
**************************************************
2022-02-24 18:38:39.702279
2022-02-24 18:38:44.267067
2022-02-24 18:39:06.585405
##################################################
722.csv
**************************************************
2022-02-24 18:39:20.017468
2022-02-24 18:39:25.113833
2022-02-24 18:39:48.929502
##################################################
723.csv
**************************************************
2022-02-24 18:40:00.756858
2022-02-24 18:40:05.714593
2022-02-24 18:40:31.280193
##################################################
724.csv
**************************************************
2022-02-24 18:40:46.298013
2022-02-24 18:40:51.055285
2022-02-24 18:41:15.090978
##################################################
725.csv
**************************************************
2022-02-24 18:41:29.333870
2022-02-24 18:41:34.179905
2022-02-24 18:41:58.171715
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d7\d73'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
731.csv
**************************************************
2022-02-24 18:45:08.656076
2022-02-24 18:45:13.169999
2022-02-24 18:45:34.315427
##################################################
732.csv
**************************************************
2022-02-24 18:45:45.741873
2022-02-24 18:45:49.649399
2022-02-24 18:46:08.713394
##################################################
733.csv
**************************************************
2022-02-24 18:46:21.546058
2022-02-24 18:46:25.471578
2022-02-24 18:46:32.169672
##################################################
734.csv
**************************************************
2022-02-24 18:46:37.567216
2022-02-24 18:46:41.753022
2022-02-24 18:46:48.696418
##################################################
735.csv
**************************************************
2022-02-24 18:46:54.081011
2022-02-24 18:46:58.424391
2022-02-24 18:47:20.385641
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d7\d74'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
741.csv
**************************************************
2022-02-24 18:49:52.798853
2022-02-24 18:49:57.459384
2022-02-24 18:50:04.780795
##################################################
742.csv
**************************************************
2022-02-24 18:50:08.790068
2022-02-24 18:50:13.104526
2022-02-24 18:50:35.718023
##################################################
743.csv
**************************************************
2022-02-24 18:50:50.262111
2022-02-24 18:50:55.385403
2022-02-24 18:51:20.843291
##################################################
744.csv
**************************************************
2022-02-24 18:51:35.745421
2022-02-24 18:51:40.645312
2022-02-24 18:51:48.220076
##################################################
745.csv
**************************************************
2022-02-24 18:51:53.269534
2022-02-24 18:52:00.553048
2022-02-24 18:52:26.324097
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d7\d75'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
751.csv
**************************************************
2022-02-24 18:55:16.260939
2022-02-24 18:55:21.409167
2022-02-24 18:55:45.249837
##################################################
752.csv
**************************************************
2022-02-24 18:55:59.646320
2022-02-24 18:56:04.385639
2022-02-24 18:56:28.157039
##################################################
753.csv
**************************************************
2022-02-24 18:56:44.392601
2022-02-24 18:56:49.402198
2022-02-24 18:57:13.909628
##################################################
754.csv
**************************************************
2022-02-24 18:57:25.915508
2022-02-24 18:57:31.128560
2022-02-24 18:57:58.661895
##################################################
755.csv
**************************************************
2022-02-24 18:58:11.316038
2022-02-24 18:58:16.678692
2022-02-24 18:58:42.635244
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d7\d76'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
761.csv
**************************************************
2022-02-24 19:01:51.990769
2022-02-24 19:01:56.677230
2022-02-24 19:02:21.753689
##################################################
762.csv
**************************************************
2022-02-24 19:02:35.098303
2022-02-24 19:02:40.079974
2022-02-24 19:03:07.479019
##################################################
763.csv
**************************************************
2022-02-24 19:03:24.094218
2022-02-24 19:03:28.930279
2022-02-24 19:03:53.927340
##################################################
764.csv
**************************************************
2022-02-24 19:04:07.133731
2022-02-24 19:04:12.306890
2022-02-24 19:04:39.795170
##################################################
765.csv
**************************************************
2022-02-24 19:04:56.111558
2022-02-24 19:05:01.262769
2022-02-24 19:05:30.157653
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d7\d77'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
771.csv
**************************************************
2022-02-24 19:09:00.215007
2022-02-24 19:09:04.705990
2022-02-24 19:09:12.681114
##################################################
772.csv
**************************************************
2022-02-24 19:09:16.823032
2022-02-24 19:09:21.868707
2022-02-24 19:09:46.837727
##################################################
773.csv
**************************************************
2022-02-24 19:10:05.433973
2022-02-24 19:10:10.495431
2022-02-24 19:10:35.041757
##################################################
774.csv
**************************************************
2022-02-24 19:10:50.522338
2022-02-24 19:10:55.267643
2022-02-24 19:11:21.295518
##################################################
775.csv
**************************************************
2022-02-24 19:11:37.469241
2022-02-24 19:11:42.387085
2022-02-24 19:12:06.647178
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d7\d78'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
781.csv
**************************************************
2022-02-24 19:15:19.656780
2022-02-24 19:15:24.136794
2022-02-24 19:15:47.087876
##################################################
782.csv
**************************************************
2022-02-24 19:16:01.543200
2022-02-24 19:16:06.287570
2022-02-24 19:16:30.095518
##################################################
783.csv
**************************************************
2022-02-24 19:16:44.050179
2022-02-24 19:16:49.035840
2022-02-24 19:17:14.245392
##################################################
784.csv
**************************************************
2022-02-24 19:17:26.968351
2022-02-24 19:17:32.286653
2022-02-24 19:17:56.575645
##################################################
785.csv
**************************************************
2022-02-24 19:18:08.841828
2022-02-24 19:18:13.951157
2022-02-24 19:18:40.779380
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d7\d79'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
791.csv
**************************************************
2022-02-24 19:21:35.746256
2022-02-24 19:21:40.068692
2022-02-24 19:21:47.646417
##################################################
792.csv
**************************************************
2022-02-24 19:21:52.695908
2022-02-24 19:21:57.791275
2022-02-24 19:22:23.254149
##################################################
793.csv
**************************************************
2022-02-24 19:22:35.078513
2022-02-24 19:22:39.906595
2022-02-24 19:23:04.414047
##################################################
794.csv
**************************************************
2022-02-24 19:23:15.772636
2022-02-24 19:23:20.521929
2022-02-24 19:23:27.672797
##################################################
795.csv
**************************************************
2022-02-24 19:23:33.591960
2022-02-24 19:23:38.484870
2022-02-24 19:23:45.670671
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d7\d710'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7101.csv
**************************************************
2022-02-24 19:25:29.348255
2022-02-24 19:25:34.514433
2022-02-24 19:25:59.749915
##################################################
7102.csv
**************************************************
2022-02-24 19:26:11.795688
2022-02-24 19:26:17.684932
2022-02-24 19:26:44.058371
##################################################
7103.csv
**************************************************
2022-02-24 19:26:56.421292
2022-02-24 19:27:01.737069
2022-02-24 19:27:26.464931
##################################################
7104.csv
**************************************************
2022-02-24 19:27:38.514670
2022-02-24 19:27:43.680849
2022-02-24 19:28:11.856465
##################################################
7105.csv
**************************************************
2022-02-24 19:28:24.486673
2022-02-24 19:28:29.947064
2022-02-24 19:28:56.601749
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d7\d711'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7111.csv
**************************************************
2022-02-24 19:32:14.293399
2022-02-24 19:32:19.177334
2022-02-24 19:32:45.865928
##################################################
7112.csv
**************************************************
2022-02-24 19:33:02.689915
2022-02-24 19:33:08.143324
2022-02-24 19:33:36.451586
##################################################
7113.csv
**************************************************
2022-02-24 19:33:49.818823
2022-02-24 19:33:55.466711
2022-02-24 19:34:21.104119
##################################################
7114.csv
**************************************************
2022-02-24 19:34:36.279517
2022-02-24 19:34:41.002880
2022-02-24 19:35:04.879998
##################################################
7115.csv
**************************************************
2022-02-24 19:35:20.301736
2022-02-24 19:35:24.896443
2022-02-24 19:35:47.630617
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d7\d712'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7121.csv
**************************************************
2022-02-24 19:38:56.455417
2022-02-24 19:39:01.354314
2022-02-24 19:39:25.944519
##################################################
7122.csv
**************************************************
2022-02-24 19:39:37.570414
2022-02-24 19:39:42.490252
2022-02-24 19:40:07.452465
##################################################
7123.csv
**************************************************
2022-02-24 19:40:22.776465
2022-02-24 19:40:27.791077
2022-02-24 19:40:52.733316
##################################################
7124.csv
**************************************************
2022-02-24 19:41:04.165729
2022-02-24 19:41:09.229206
2022-02-24 19:41:34.464697
##################################################
7125.csv
**************************************************
2022-02-24 19:41:48.384444
2022-02-24 19:41:53.222505
2022-02-24 19:42:17.478580
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d7\d713'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7131.csv
**************************************************
2022-02-24 19:45:24.609912
2022-02-24 19:45:29.680346
2022-02-24 19:45:54.693507
##################################################
7132.csv
**************************************************
2022-02-24 19:46:09.553666
2022-02-24 19:46:15.100825
2022-02-24 19:46:42.012842
##################################################
7133.csv
**************************************************
2022-02-24 19:46:56.062232
2022-02-24 19:47:01.671227
2022-02-24 19:47:09.315772
##################################################
7134.csv
**************************************************
2022-02-24 19:47:13.582358
2022-02-24 19:47:19.180387
2022-02-24 19:47:26.978517
##################################################
7135.csv
**************************************************
2022-02-24 19:47:31.300951
2022-02-24 19:47:36.558885
2022-02-24 19:47:44.174508
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d7\d714'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7141.csv
**************************************************
2022-02-24 19:49:46.386530
2022-02-24 19:49:51.461951
2022-02-24 19:50:16.482010
##################################################
7142.csv
**************************************************
2022-02-24 19:50:31.836928
2022-02-24 19:50:36.609160
2022-02-24 19:51:00.247914
##################################################
7143.csv
**************************************************
2022-02-24 19:51:15.421318
2022-02-24 19:51:20.639357
2022-02-24 19:51:46.749534
##################################################
7144.csv
**************************************************
2022-02-24 19:52:00.754031
2022-02-24 19:52:05.993014
2022-02-24 19:52:32.130084
##################################################
7145.csv
**************************************************
2022-02-24 19:52:46.015933
2022-02-24 19:52:51.104319
2022-02-24 19:53:16.180228
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d7\d715'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7151.csv
**************************************************
2022-02-24 19:56:14.025405
2022-02-24 19:56:19.044001
2022-02-24 19:56:44.159802
##################################################
7152.csv
**************************************************
2022-02-24 19:56:55.836539
2022-02-24 19:57:00.692546
2022-02-24 19:57:24.412088
##################################################
7153.csv
**************************************************
2022-02-24 19:57:35.935256
2022-02-24 19:57:41.095475
2022-02-24 19:58:06.173354
##################################################
7154.csv
**************************************************
2022-02-24 19:58:17.611750
2022-02-24 19:58:22.753993
2022-02-24 19:58:47.909688
##################################################
7155.csv
**************************************************
2022-02-24 19:59:01.668897
2022-02-24 19:59:06.666504
2022-02-24 19:59:31.190911
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d7\d716'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7161.csv
**************************************************
2022-02-24 20:02:15.967032
2022-02-24 20:02:20.249574
2022-02-24 20:02:41.949516
##################################################
7162.csv
**************************************************
2022-02-24 20:02:52.833396
2022-02-24 20:02:57.689403
2022-02-24 20:03:21.649327
##################################################
7163.csv
**************************************************
2022-02-24 20:03:34.871922
2022-02-24 20:03:39.661136
2022-02-24 20:04:04.110694
##################################################
7164.csv
**************************************************
2022-02-24 20:04:17.477931
2022-02-24 20:04:22.361899
2022-02-24 20:04:46.536185
##################################################
7165.csv
**************************************************
2022-02-24 20:04:59.896439
2022-02-24 20:05:04.247825
2022-02-24 20:05:25.998637
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d7\d717'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7171.csv
**************************************************
2022-02-24 20:08:33.452373
2022-02-24 20:08:37.875539
2022-02-24 20:09:00.394325
##################################################
7172.csv
**************************************************
2022-02-24 20:09:12.270516
2022-02-24 20:09:19.784655
2022-02-24 20:09:46.841266
##################################################
7173.csv
**************************************************
2022-02-24 20:10:01.611747
2022-02-24 20:10:06.870677
2022-02-24 20:10:34.906720
##################################################
7174.csv
**************************************************
2022-02-24 20:10:50.870147
2022-02-24 20:10:56.384393
2022-02-24 20:11:23.065009
##################################################
7175.csv
**************************************************
2022-02-24 20:11:37.352804
2022-02-24 20:11:42.575808
2022-02-24 20:12:11.084533
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d7\d718'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7181.csv
**************************************************
2022-02-24 20:15:21.871593
2022-02-24 20:15:26.510182
2022-02-24 20:15:53.718440
##################################################
7182.csv
**************************************************
2022-02-24 20:16:08.646499
2022-02-24 20:16:14.107888
2022-02-24 20:16:40.258463
##################################################
7183.csv
**************************************************
2022-02-24 20:16:56.771252
2022-02-24 20:17:01.924493
2022-02-24 20:17:27.373376
##################################################
7184.csv
**************************************************
2022-02-24 20:17:46.020034
2022-02-24 20:17:51.543257
2022-02-24 20:18:17.958583
##################################################
7185.csv
**************************************************
2022-02-24 20:18:33.731382
2022-02-24 20:18:38.762921
2022-02-24 20:19:04.009373
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d7\d719'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7191.csv
**************************************************
2022-02-24 20:22:23.038866
2022-02-24 20:22:27.753743
2022-02-24 20:22:51.908097
##################################################
7192.csv
**************************************************
2022-02-24 20:23:04.047618
2022-02-24 20:23:08.735077
2022-02-24 20:23:32.298035
##################################################
7193.csv
**************************************************
2022-02-24 20:23:44.061575
2022-02-24 20:23:49.119079
2022-02-24 20:24:15.825626
##################################################
7194.csv
**************************************************
2022-02-24 20:24:27.304914
2022-02-24 20:24:32.876007
2022-02-24 20:24:59.753934
##################################################
7195.csv
**************************************************
2022-02-24 20:25:11.494522
2022-02-24 20:25:16.459269
2022-02-24 20:25:42.174460
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d7\d720'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
7201.csv
**************************************************
2022-02-24 20:29:08.531424
2022-02-24 20:29:13.139096
2022-02-24 20:29:21.573531
##################################################
7202.csv
**************************************************
2022-02-24 20:29:28.944809
2022-02-24 20:29:34.370314
2022-02-24 20:30:01.234418
##################################################
7203.csv
**************************************************
2022-02-24 20:30:17.636551
2022-02-24 20:30:23.225580
2022-02-24 20:30:33.317133
##################################################
7204.csv
**************************************************
2022-02-24 20:30:37.012265
2022-02-24 20:30:42.225324
2022-02-24 20:31:08.120015
##################################################
7205.csv
**************************************************
2022-02-24 20:31:22.140504
2022-02-24 20:31:26.966592
2022-02-24 20:31:52.606991
########################################